In [1]:
#!pip install sqlalchemy==1.3.9
#!pip install ipython-sql # SQL magic function
#!pip install sqlalchemy --upgrade
#!pip install --upgrade pip

In [1]:
# import csv, sqlite3

In [2]:
%reload_ext sql
%sql sqlite:///flight_data.db

In [3]:
import csv, sqlite3
con = sqlite3.connect("flight_data.db")
cur = con.cursor()

In [5]:
import pandas as pd
df = pd.read_csv("Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False, method="multi")

101

In [6]:
%sql SELECT * FROM SPACEXTBL LIMIT 10

 * sqlite:///flight_data.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing _Outcome
04-06-2010,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
08-12-2010,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
22-05-2012,07:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
08-10-2012,00:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
01-03-2013,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt
29-09-2013,16:00:00,F9 v1.1 B1003,VAFB SLC-4E,CASSIOPE,500,Polar LEO,MDA,Success,Uncontrolled (ocean)
03-12-2013,22:41:00,F9 v1.1,CCAFS LC-40,SES-8,3170,GTO,SES,Success,No attempt
06-01-2014,22:06:00,F9 v1.1,CCAFS LC-40,Thaicom 6,3325,GTO,Thaicom,Success,No attempt
18-04-2014,19:25:00,F9 v1.1,CCAFS LC-40,SpaceX CRS-3,2296,LEO (ISS),NASA (CRS),Success,Controlled (ocean)
14-07-2014,15:15:00,F9 v1.1,CCAFS LC-40,OG2 Mission 1 6 Orbcomm-OG2 satellites,1316,LEO,Orbcomm,Success,Controlled (ocean)


### Display the names of the unique launch sites  in the space mission

In [7]:
%sql SELECT DISTINCT Launch_Site FROM SPACEXTBL

 * sqlite:///flight_data.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


### Display 5 records where launch sites begin with the string 'CCA'

In [8]:
%sql SELECT Launch_Site FROM SPACEXTBL WHERE Launch_Site LIKE 'CCA%' LIMIT 5

 * sqlite:///flight_data.db
Done.


Launch_Site
CCAFS LC-40
CCAFS LC-40
CCAFS LC-40
CCAFS LC-40
CCAFS LC-40


### Display average payload mass carried by booster version F9 v1.1

In [9]:
%sql SELECT AVG(PAYLOAD_MASS__KG_) FROM SPACEXTBL WHERE Booster_Version LIKE 'F9 v1.1%'

 * sqlite:///flight_data.db
Done.


AVG(PAYLOAD_MASS__KG_)
2534.6666666666665


### List the date when the first succesful landing outcome in ground pad was acheived.

In [10]:
%%sql 
SELECT MIN(Date) 
FROM SPACEXTBL 
WHERE [Landing _Outcome] LIKE 'Success (ground pad)'

 * sqlite:///flight_data.db
Done.


MIN(Date)
01-05-2017


### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000

In [11]:
#%sql SELECT DISTINCT [Landing _Outcome] FROM SPACEXTBL

In [12]:
%%sql
SELECT DISTINCT Booster_Version
FROM SPACEXTBL
WHERE PAYLOAD_MASS__KG_ > 4000
AND PAYLOAD_MASS__KG_ < 6000
AND [Landing _Outcome] LIKE 'Success (drone ship)'

 * sqlite:///flight_data.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


### List the total number of successful and failure mission outcomes

In [13]:
%%sql
SELECT Mission_Outcome, COUNT(Mission_Outcome)
FROM SPACEXTBL
GROUP BY Mission_Outcome

 * sqlite:///flight_data.db
Done.


Mission_Outcome,COUNT(Mission_Outcome)
Failure (in flight),1
Success,98
Success,1
Success (payload status unclear),1


### List the   names of the booster_versions which have carried the maximum payload mass. Use a subquery

In [14]:
%%sql 
SELECT DISTINCT Booster_Version 
FROM SPACEXTBL
WHERE PAYLOAD_MASS__KG_ = (SELECT MAX(PAYLOAD_MASS__KG_) FROM SPACEXTBL)

 * sqlite:///flight_data.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


### List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

**Note: SQLLite does not support monthnames. So you need to use  substr(Date, 4, 2) as month to get the months and substr(Date,7,4)='2015' for year.**

In [15]:
%%sql
SELECT 
    substr(Date, 4, 2) as month,
    substr(Date,7,4) as year,
    [Landing _Outcome],
    Booster_Version,
    Launch_Site
FROM SPACEXTBL 
WHERE substr(Date,7,4)='2015'
AND [Landing _Outcome] LIKE 'Failure (drone ship)'

 * sqlite:///flight_data.db
Done.


month,year,Landing _Outcome,Booster_Version,Launch_Site
01,2015,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
04,2015,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


### Rank the  count of  successful landing_outcomes between the date 04-06-2010 and 20-03-2017 in descending order.

In [16]:
%%sql 
SELECT 
Booster_Version,
[Landing _Outcome],
substr(Date,7,4) || '-' || substr(Date, 4, 2) || '-' || substr(Date, 1, 2)AS YMD
FROM SPACEXTBL 
WHERE [Landing _Outcome] LIKE 'Success%'
AND YMD > '2010-06-04'
AND YMD < '2017-03-20'
ORDER BY YMD DESC 

 * sqlite:///flight_data.db
Done.


Booster_Version,Landing _Outcome,YMD
F9 FT B1031.1,Success (ground pad),2017-02-19
F9 FT B1029.1,Success (drone ship),2017-01-14
F9 FT B1026,Success (drone ship),2016-08-14
F9 FT B1025.1,Success (ground pad),2016-07-18
F9 FT B1023.1,Success (drone ship),2016-05-27
F9 FT B1022,Success (drone ship),2016-05-06
F9 FT B1021.1,Success (drone ship),2016-04-08
F9 FT B1019,Success (ground pad),2015-12-22


In [17]:
%%sql 
SELECT max(Date) from SPACEXTBL

 * sqlite:///flight_data.db
Done.


max(Date)
31-01-2018
